In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import time
import os
import re
from itertools import chain
from transformers import BertTokenizer, BertForMaskedLM
from IPython.display import clear_output
import pandas as pd

PRETRAINED_MODEL_NAME = "bert-base-chinese" #英文pretrain(不區分大小寫)
print(torch.__version__)

/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu-gpu/anaconda3/envs/trainEnv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ubuntu-gpu/anaconda3/e

1.5.0


In [2]:
# get pre-train tokenizer
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME)
vocab = tokenizer.vocab
print("dict size", len(vocab))

# see some token and index mapping
import random
random_tokens = random.sample(list(vocab), 10)
random_ids = [vocab[t] for t in random_tokens]

print("{0:20}{1:15}".format("token", "index"))
print("-" * 25)
for t, id in zip(random_tokens, random_ids): #隨便看幾個字
    print("{0:15}{1:10}".format(t, id))

dict size 21128
token               index          
-------------------------
##tics              11130
cream               11494
##皱                 17708
##喫                 14661
url                  8654
##ｐ                 10748
##趕                 19691
##息                 15679
##ナー                12005
隴                    7404


In [10]:
train = pd.read_csv('train.csv')
val = pd.read_csv('validation.csv')
total = pd.concat([train, val])
total = total.reset_index(drop=True)
total_0 = total[total["Label"] == 0]
total_1 = total[total["Label"] == 1]

In [11]:
# Label_1
num_1 = total_1.shape[0]
random_1 = random.sample(range(num_1), num_1)
train_1 = total_1.iloc[random_1[:108]]
test_1 = total_1.iloc[random_1[108:135]]
# Label_0
num_0 = total_0.shape[0]
random_0 = random.sample(range(num_0), num_0)
train_0 = total_0.iloc[random_0[:400]]
test_0 = total_0.iloc[random_0[400:500]]
# train_val_1
train_val_1 = train_1.shape[0]
random_train_val = random.sample(range(train_val_1), train_val_1)
train__1 = train_1.iloc[random_train_val[:97]]
val_1 = train_1.iloc[random_train_val[97:108]]
# train_val_0
train_val_0 = train_0.shape[0]
random_train_val = random.sample(range(train_val_0), train_val_0)
train__0 = train_0.iloc[random_train_val[:360]]
val_0 = train_0.iloc[random_train_val[360:400]]
# train_val
train = pd.concat([train__1, train__0])
train = train.reset_index(drop=True)
val = pd.concat([val_1, val_0])
val = val.reset_index(drop=True)
# test
test = pd.concat([test_1, test_0])
test = test.reset_index(drop=True)

In [19]:
# 剔除過長的樣本以避免 BERT 無法將整個輸入序列放入記憶體不多的 GPU
MAX_LENGTH = 250
train = train[~(train.Content.apply(lambda x : len(x)) > MAX_LENGTH)]
test = test[~(test.Content.apply(lambda x : len(x)) > MAX_LENGTH)]

In [15]:
text_a = train['Content']

In [18]:
text_a

1      每天都很空洞不知道自己到底為什麼下班之後更是常常莫名想哭莫名的不想動莫名的一直自己掐住喉嚨到...
2      一直撐著不對自己下手但又一直痛苦著好痛好痛好難過我已經不行了遺書一直寫了又刪寫了又刪我撐不下...
3      憂鬱症到現在也大概兩年多了第一次吃藥因為藥太重出現混亂的狀態很快就被要求休學回老家休息後來還...
4      一下班強迫症跟憂鬱又大發作明明只是第二天上班步調好快我跟不上根本也都沒準時下班又沾東沾西要死...
5      系館旁有一小片樹林，我們稱之為黑森林。黑森林並不黑，隔壁是個水池，裡面有烏龜在游泳。我跟學妹...
                             ...                        
449    好痛苦， 每天從晚上12點睡到晚上6點， 睡十幾個小時， 醒來沒多久又要睡了。 做不了什麼事...
453    https://i.imgur.com/lJznqj4.jpg心理狀態越來越糟也許下次回診該...
454    年輕時就已經一直失業了現在都快步了中年了，年紀愈來愈大，失業時工作愈難找明明知道不能挑工作，...
455    四月，我因為自殺未遂進了醫院。五月，六月，我都在精神科病房度過。精神科病房規定不能攜帶任何電...
456    哈囉大家好我是精靈本期我將會介紹世界上數一數二知名的遊戲公司—MOJANG就是開發MINEC...
Name: Content, Length: 294, dtype: object

In [57]:
def token(text):
    # 建立第一個句子的 BERT tokens
    for i in range(len(text)):
        tokens_a = tokenizer.tokenize(text[i])
        len_a = len(word_pieces)

In [58]:
token(text_a)

In [59]:
# 將整個 token 序列轉換成索引序列
ids = tokenizer.convert_tokens_to_ids(word_pieces)
tokens_tensor = torch.tensor(ids)
tokens_tensor

tensor([ 101, 1333, 3315,  100, 2769,  100,  100, 2769,  100,  677, 4638,  100,
         100, 2769,  100,  100,  100, 3315,  100,  100,  100,  100,  100,  100,
         100, 2769, 4638,  100, 7279,  100,  100,  100,  677,  100,  722, 2527,
         100,  100,  679,  100,  100,  100,  671,  100,  752,  100,  100, 2769,
         100, 4638,  100,  100,  100,  753, 4638, 5739, 3152, 1366,  100,  100,
         100,  100,  100,  100,  677, 1398,  100, 6963,  100,  671,  100,  100,
        2695,  100, 1921, 4638,  100,  100,  100,  100,  677,  100, 6963,  100,
         100,  100,  100,  100,  100,  679,  100, 4638,  738,  100,  100, 2769,
         100,  100,  100, 2399,  100,  782, 6963,  100,  100,  100,  100,  100,
         100, 1922,  100,  100,  100, 2769,  671,  100,  100,  100,  100,  100,
         100,  100,  100,  100,  100,  100,  100,  100,  671,  100,  100,  100,
        4638, 5739, 3152, 2527,  100, 2769,  100,  100,  100,  100,  100,  100,
         100,  100,  752,  100,  100,  1